In [1]:
import h5py
import numpy as np

In [2]:
from e2c_train import create_e2c

from keras import backend as K
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [3]:
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'

output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/' # load model data

case_name = '9w_bhp_rate'

# suffix = '_single_out_rel_2'
# suffix = '_fix_wl_rel_1'
case_suffix = '_fix_wl_rel_1' # the dataset being evaluated here
# model_suffix = '_single_out_rel_3' # the dataset used to train the model
model_suffix = '_fix_wl_rel_1'
train_suffix = '_with_p'

eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix + '_n2200_dt20day_nt22_nrun100.mat'

state_file = case_name + '_train_n_400_full'
ctrl_file = case_name + '_norm_bhps_n_400'

state_data = state_file + case_suffix + '.mat'
ctrl_data = ctrl_file + case_suffix + '.mat'

In [4]:
latent_dim, u_dim = 50, 9
input_shape = (60, 60, 2) # change from _with_p to _no_p
encoder, decoder, transition, sampler = create_e2c(latent_dim, u_dim, input_shape)

num_train, latent_dim, learning_rate, epoch = 6600, 50, 1e-4, 10

In [5]:
encoder.load_weights(output_dir + 'e2c_encoder_'+case_name+model_suffix+train_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))
decoder.load_weights(output_dir + 'e2c_decoder_'+case_name+model_suffix+train_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))
transition.load_weights(output_dir + 'e2c_transition_'+case_name+model_suffix+train_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))

## One step prediction

In [6]:
hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
# state_t1_eval = np.array(hf_r.get('state_t1'))
# print(list(hf_r.keys()))
# state_t_eval = np.array(hf_r.get('sat_t'))
# state_t1_eval = np.array(hf_r.get('sat_t1'))
# bhp_eval = np.array(hf_r.get('bhp'))
hf_r.close()

In [8]:
num_eval = 20 # pick 20 out of 2200 evals
state_t_eval = state_t_eval[:num_eval, ...]
# state_t1_eval = state_t1_eval[:num_eval, ...]
# bhp_eval = bhp_eval[:num_eval, ...]

In [9]:
state_t_eval.shape

(20, 60, 60, 2)

In [ ]:
hf_r = h5py.File(data_dir + state_file + case_suffix + '.mat', 'r')
sat_t = np.array(hf_r.get('sat'))
pres_t = np.array(hf_r.get('pres'))
print(sat_t.shape)
print(pres_t.shape)

In [ ]:
sat = sat.T.reshape((400, 100, 3600))
pres = pres.T.reshape(400,100,3600)
print("sat shape:{}".format(sat.shape))
print("pres shape:{}".format(pres.shape))

In [13]:
[xi_t_eval, _] = encoder.predict(state_t_eval)

In [14]:
xi_t_eval.shape

(20, 50)

In [ ]:
np.savetxt('data/xi_t_eval_2200.txt',xi_t_eval)

In [ ]:
import matplotlib as plt
%matplotlib inline
plt.plot(t_step)